## Example for binning and correlating data

This example shows how to use some pre-built methods to bin two sets of data in time so that they can be compared to look for correlations.

In [104]:
# Importing local python package
import sys
sys.path.append(r'C:/Users/ahanks/E11-git/Activities/')
sys.path.append('.')
import LabMethods
import importlib
importlib.reload(LabMethods)

<module 'LabMethods' from 'C:\\Users\\ahanks\\E11-git\\Activities\\LabMethods.py'>

In [95]:
# importing other python tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import requests

### Import and prepare data from our Etcheverry sensor system

We will use radiation data like we've seen before, but we are also going to take a look at some weather data:
* "https://radwatch.berkeley.edu/test/tmp/dosenet/etch_roof.csv"
* "https://radwatch.berkeley.edu/test/tmp/dosenet/etch_roof_weather.csv"

#### Make methods for importing this data so we don't have to write all of the steps every time we want to import new data.

In [96]:
def get_web_data(url):
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    s = requests.get(url, headers=header).text
    data = pd.read_csv(io.StringIO(s))
    return data

In [ ]:
url = "https://radwatch.berkeley.edu/test/tmp/dosenet/etch_roof_weather.csv"
weather_data = get_web_data(url)
weather_data

#### Cut down the data - focus on the weather data for now
* Keep only the relevant columns (time, temperature, humidity, pressure)
* Select only the last month

In [ ]:
weather_data_cut = weather_data.loc[:,"deviceTime_local":"humidity"]
time_mask = weather_data_cut['deviceTime_local']>'2018-03-22 00:00:00:-07:00'
weather_month = weather_data_cut[time_mask]
weather_month

#### Correlating data from same sensor

In [ ]:
plt.scatter(weather_month['humidity'].values,weather_month['temperature'].values)
plt.ylabel("Temperature")
plt.xlabel("Humidity")
plt.show()

### Quantifying this relationship

We can define a correlation coeefficient ($\rho$, or $r_{xy}$ to describe how correlated these two variables, Temperature(y) and Humidity(x), are, taking into account the **variance** of in each variable, and the **covariance** - how much the two variable vary together.

$r_{xy} = \frac{Cov(x,y)}{\sigma_{x}\sigma_{y}}$

$r_{xy} = \frac{\sum_{i=0}^{N}(x_{i}-\mu_{x})(y_{i}-\mu_{y})}{\sqrt\sum_{i=0}^{N}(x_{i}-\mu_{x})^{2}\sqrt\sum_{i=0}^{N}(y_{i}-\mu_{y})^{2}}$

This is known as the Pearson correlation - and there is already a statistics library for python that defines this method as `pearsonr`. You can then call this method to look at the correlation between two sets of data (data1 and data2):

    import scipy.stats
    scipy.stats.pearsonr(data1, data2)


In [ ]:
import scipy.stats

r, p = scipy.stats.pearsonr(weather_month['humidity'].values, weather_month['temperature'].values)
print("Humidity vs Temp: corr. coeff. = {}, p-value = {}".format(r**2, p))

r_press, p_press = scipy.stats.pearsonr(weather_month['humidity'].values, weather_month['pressure'].values)

r_sqr = r_press**2
print("Humidity vs Pressure: corr. coeff. = {}, p-value = {}".format(r_press, p_press))

###### NOTE 1: The square of this correlation coefficient is the fraction (%) of all variation in each data set that can be explained by the variation in the other data set.

###### NOTE 2: The second measure of correlation is the statistical significance of the measured correlation coefficient.

The significance can be estimated by the probability that a randomized sampling of the x and y would produce the same correlation coefficient:

1. Randomize the x and y data sets (this will break any real correlation)
2. Measure the correlation for this new set of x and y measurements
3. Do this many times - the probability of getting a correlation strength greater than that measured in the original data indicates how confident we can be that the correlation is real


In [ ]:
import random

rand_temp = np.copy(weather_month['temperature'].values)
print(rand_temp)
random.shuffle(rand_temp)
print(rand_temp)

rand_hum = np.copy(weather_month['humidity'].values)
print(rand_hum)
random.shuffle(rand_hum)
print(rand_hum)

scipy.stats.pearsonr(weather_month['humidity'].values, weather_month['temperature'].values)
scipy.stats.pearsonr(rand_hum, rand_temp)

### Explore the correlation between the two types of radiation data we have available (8pts)
For this we will look at the "local" copies of this data because of the large size of the d3s data set.
* etch_roof.csv
* etch_roof_d3s.csv

These have been downloaded with a path of this format: 

    C:/Users/<user-name>/<path-to-file>/<filename>

In [114]:
#url = "https://radwatch.berkeley.edu/test/tmp/dosenet/etch_roof.csv"
#data1 = get_web_data(url)

data1 = pd.read_csv(r"C:/Users/ahanks/DoseNet/summer-work/data/etch_roof.csv")
data1

,deviceTime_utc,deviceTime_local,deviceTime_unix,cpm,cpmError,error_flag
0,2015-11-20 02:27:13+00:00,2015-11-19 18:27:13-08:00,1.447986e+09,4.0,0.894427,NaN
1,2015-11-20 02:32:13+00:00,2015-11-19 18:32:13-08:00,1.447987e+09,2.8,0.748331,NaN
2,2015-11-20 02:37:15+00:00,2015-11-19 18:37:15-08:00,1.447987e+09,3.4,0.824621,NaN
3,2015-11-20 02:42:15+00:00,2015-11-19 18:42:15-08:00,1.447987e+09,2.4,0.692820,NaN
4,2015-11-20 02:47:16+00:00,2015-11-19 18:47:16-08:00,1.447988e+09,3.4,0.824621,NaN
...,...,...,...,...,...,...
394491,2022-03-24 02:45:35+0000,2022-03-23 19:45:35-0700,1.648090e+09,2.0,0.632456,0.0
394492,2022-03-24 02:50:35+0000,2022-03-23 19:50:35-0700,1.648090e+09,3.2,0.800000,0.0
394493,2022-03-24 02:55:35+0000,2022-03-23 19:55:35-0700,1.648091e+09,3.0,0.774597,0.0
394494,2022-03-24 03:00:35+0000,2022-03-23 20:00:35-0700,1.648091e+09,2.0,0.632456,0.0


In [115]:
#url = "https://radwatch.berkeley.edu/test/tmp/dosenet/etch_roof_weather.csv"
#data2 = get_web_data(url)

#data2 = pd.read_csv(r"C:/Users/ahanks/DoseNet/summer-work/data/etch_roof_weather.csv")
data2 = pd.read_csv(r"C:/Users/ahanks/DoseNet/summer-work/data/etch_roof_d3s.csv")
data2

,deviceTime_utc,deviceTime_local,deviceTime_unix,cpm,cpmError,keV_per_ch,0,1,2,3,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,error_flag
0,2017-03-15 20:50:59+00:00,2017-03-15 13:50:59-07:00,1.489611e+09,2121.6,20.599029,2.50,0,0,0,0,...,0,0,0,0,0,0,0,0,71,NaN
1,2017-03-15 20:55:59+00:00,2017-03-15 13:55:59-07:00,1.489611e+09,2149.8,20.735477,2.50,0,0,0,0,...,0,0,0,0,0,0,0,0,55,NaN
2,2017-03-15 21:01:00+00:00,2017-03-15 14:01:00-07:00,1.489612e+09,2192.2,20.938959,2.50,0,0,0,0,...,0,0,0,1,0,0,0,0,61,NaN
3,2017-03-15 21:14:54+00:00,2017-03-15 14:14:54-07:00,1.489612e+09,2145.8,20.716177,2.50,0,0,0,0,...,0,0,0,0,0,1,0,0,78,NaN
4,2017-03-15 21:19:54+00:00,2017-03-15 14:19:54-07:00,1.489613e+09,2119.6,20.589318,2.50,0,0,0,0,...,1,0,0,0,0,0,0,0,78,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218286,2020-10-17 00:07:44+0000,2020-10-16 17:07:44-0700,1.602893e+09,2352.6,21.691473,2.57,0,0,0,0,...,0,1,0,0,0,0,0,0,63,0.0
218287,2020-10-17 00:12:44+0000,2020-10-16 17:12:44-0700,1.602894e+09,2307.6,21.483017,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,62,0.0
218288,2020-10-17 00:17:44+0000,2020-10-16 17:17:44-0700,1.602894e+09,2322.6,21.552726,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,59,0.0
218289,2020-10-17 00:22:44+0000,2020-10-16 17:22:44-0700,1.602894e+09,2344.2,21.652713,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,58,0.0


In [116]:
time_mask = (data1['deviceTime_local']>'2020-09-17 00:00:00:-07:00') & (data1['deviceTime_local']<'2020-10-17 00:00:00:-07:00')
data1_cut = data1[time_mask]
data1_cut

,deviceTime_utc,deviceTime_local,deviceTime_unix,cpm,cpmError,error_flag
330413,2020-09-17 07:00:42+0000,2020-09-17 00:00:42-0700,1.600326e+09,1.6,0.565685,0.0
330414,2020-09-17 07:05:42+0000,2020-09-17 00:05:42-0700,1.600326e+09,3.6,0.848528,0.0
330415,2020-09-17 07:10:42+0000,2020-09-17 00:10:42-0700,1.600327e+09,3.0,0.774597,0.0
330416,2020-09-17 07:15:42+0000,2020-09-17 00:15:42-0700,1.600327e+09,2.6,0.721110,0.0
330417,2020-09-17 07:20:42+0000,2020-09-17 00:20:42-0700,1.600327e+09,1.0,0.447214,0.0
...,...,...,...,...,...,...
339037,2020-10-17 06:35:42+0000,2020-10-16 23:35:42-0700,1.602917e+09,4.4,0.938083,0.0
339038,2020-10-17 06:40:42+0000,2020-10-16 23:40:42-0700,1.602917e+09,5.8,1.077033,0.0
339039,2020-10-17 06:45:42+0000,2020-10-16 23:45:42-0700,1.602917e+09,3.8,0.871780,0.0
339040,2020-10-17 06:50:42+0000,2020-10-16 23:50:42-0700,1.602917e+09,4.4,0.938083,0.0


In [117]:
time_mask = (data2['deviceTime_local']>'2020-09-17 00:00:00:-07:00') & (data2['deviceTime_local']<'2020-10-17 00:00:00:-07:00')
data2_cut = data2[time_mask]
data2_cut

,deviceTime_utc,deviceTime_local,deviceTime_unix,cpm,cpmError,keV_per_ch,0,1,2,3,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,error_flag
209733,2020-09-17 07:02:05+0000,2020-09-17 00:02:05-0700,1.600326e+09,2296.8,21.432685,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,68,0.0
209734,2020-09-17 07:07:05+0000,2020-09-17 00:07:05-0700,1.600326e+09,2302.6,21.459730,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,67,0.0
209735,2020-09-17 07:12:05+0000,2020-09-17 00:12:05-0700,1.600327e+09,2303.0,21.461594,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,58,0.0
209736,2020-09-17 07:17:05+0000,2020-09-17 00:17:05-0700,1.600327e+09,2305.2,21.471842,2.57,0,0,0,0,...,0,0,0,1,0,0,0,0,68,0.0
209737,2020-09-17 07:22:05+0000,2020-09-17 00:22:05-0700,1.600327e+09,2298.8,21.442015,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,68,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218286,2020-10-17 00:07:44+0000,2020-10-16 17:07:44-0700,1.602893e+09,2352.6,21.691473,2.57,0,0,0,0,...,0,1,0,0,0,0,0,0,63,0.0
218287,2020-10-17 00:12:44+0000,2020-10-16 17:12:44-0700,1.602894e+09,2307.6,21.483017,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,62,0.0
218288,2020-10-17 00:17:44+0000,2020-10-16 17:17:44-0700,1.602894e+09,2322.6,21.552726,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,59,0.0
218289,2020-10-17 00:22:44+0000,2020-10-16 17:22:44-0700,1.602894e+09,2344.2,21.652713,2.57,0,0,0,0,...,0,0,0,0,0,0,0,0,58,0.0


#### You may find you need to cut outliers from your data - check the time series first (not shown here)

In [118]:
# It looks like we want to cut out those outliers in the forst set of CPM data
# - in the time series we can see that there was a time when the detector was clearly malfunctioning
# - we can use our knowledge of the distribution for this, and cut based on the variance
data1_mean = np.mean(data1_cut['cpm'].values)
data1_std = np.std(data1_cut['cpm'].values)
data1_cut2 = data1_cut.copy()
data_mask = data1_cut2['cpm']<data1_mean+3*data1_std
data1_cut2 = data1_cut2[data_mask]
# zero counts also means the detector is malfunctioning
data_mask = data1_cut2['cpm']>0
data1_cut2 = data1_cut2[data_mask]


#### We will import some tools for binning this data to match dates correctly when trying to look at correlations.

In [119]:
# The rebin units are minutes, so this is making 5 minute bins
#  - for larger time intervals within each data bin, just increase this value (so hour intervals would be rebin=60)
rebin=30
data1_binned, data2_binned = LabMethods.bin_correlation_data(data1_cut2.copy(), data2_cut.copy(),rebin)

dropping a nan entry:  deviceTime_utc     2020-10-17 00:30:00
deviceTime_unix                    NaN
cpm                                NaN
cpmError                           NaN
error_flag                         NaN
Name: 1426, dtype: object deviceTime_utc     2020-10-17 00:30:00
deviceTime_unix            1.60289e+09
cpm                            2331.57
cpmError                       21.5941
keV_per_ch                        2.57
                          ...         
1020                                 0
1021                                 0
1022                                 0
1023                              57.5
error_flag                           0
Name: 1426, Length: 1030, dtype: object


In [120]:
data1_binned

,deviceTime_utc,deviceTime_unix,cpm,cpmError,error_flag
0,2020-09-17 07:30:00,1.600327e+09,2.366667,0.674992,0.0
1,2020-09-17 08:00:00,1.600329e+09,2.466667,0.694359,0.0
2,2020-09-17 08:30:00,1.600330e+09,3.100000,0.780459,0.0
3,2020-09-17 09:00:00,1.600332e+09,2.766667,0.742278,0.0
4,2020-09-17 09:30:00,1.600334e+09,2.966667,0.764326,0.0
...,...,...,...,...,...
1421,2020-10-16 22:00:00,1.602885e+09,4.200000,0.913958,0.0
1422,2020-10-16 22:30:00,1.602886e+09,4.600000,0.950722,0.0
1423,2020-10-16 23:00:00,1.602888e+09,4.866667,0.978367,0.0
1424,2020-10-16 23:30:00,1.602890e+09,6.433333,1.132818,0.0


In [121]:
data2_binned

,deviceTime_utc,deviceTime_unix,cpm,cpmError,keV_per_ch,0,1,2,3,4,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,error_flag
0,2020-09-17 07:30:00,1.600327e+09,2305.100000,21.471336,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,65.166667,0.0
1,2020-09-17 08:00:00,1.600329e+09,2305.533333,21.473291,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,63.333333,0.0
2,2020-09-17 08:30:00,1.600330e+09,2308.733333,21.488129,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,57.333333,0.0
3,2020-09-17 09:00:00,1.600332e+09,2301.300000,21.453429,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.333333,0.166667,0.0,0.000000,61.333333,0.0
4,2020-09-17 09:30:00,1.600334e+09,2307.800000,21.483796,2.57,0.0,0.0,0.0,0.0,0.0,...,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,61.833333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,2020-10-16 22:00:00,1.602885e+09,2321.433333,21.547170,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.0,0.166667,65.333333,0.0
1422,2020-10-16 22:30:00,1.602887e+09,2309.233333,21.490517,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,61.000000,0.0
1423,2020-10-16 23:00:00,1.602888e+09,2310.700000,21.497213,2.57,0.0,0.0,0.0,0.0,0.0,...,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,65.000000,0.0
1424,2020-10-16 23:30:00,1.602890e+09,2321.300000,21.546509,2.57,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.166667,0.166667,0.000000,0.000000,0.0,0.166667,64.833333,0.0


In [123]:
import pytz
pytz.utc.localize(data1_binned['deviceTime_utc'].iloc[0])

Timestamp('2020-09-17 07:30:00+0000', tz='UTC')

In [124]:
tz = pytz.timezone('America/Los_Angeles')
data1_binned['deviceTime_utc'].iloc[0].replace(tzinfo=pytz.utc).astimezone(tz)

Timestamp('2020-09-17 00:30:00-0700', tz='America/Los_Angeles')

In [125]:
data1_copy = data1_binned
data1_copy.loc[:,'deviceTime_local'] = data1_binned['deviceTime_utc']
data1_copy = data1_copy[['deviceTime_utc','deviceTime_local','deviceTime_unix']+[c for c in data1_copy if c not in ['deviceTime_utc','deviceTime_local','deviceTime_unix']]]

In [130]:
data1_final = data1_copy.copy()
tz = pytz.timezone('America/Los_Angeles')
for i in range(len(data1_final)):
    data1_final.loc[:,'deviceTime_local'].iloc[i] = data1_copy['deviceTime_utc'].iloc[i].replace(tzinfo=pytz.utc).astimezone(tz)

In [131]:
data1_final

,deviceTime_utc,deviceTime_local,deviceTime_unix,cpm,cpmError,error_flag
0,2020-09-17 07:30:00,2020-09-17 00:30:00-07:00,1.600327e+09,2.366667,0.674992,0.0
1,2020-09-17 08:00:00,2020-09-17 01:00:00-07:00,1.600329e+09,2.466667,0.694359,0.0
2,2020-09-17 08:30:00,2020-09-17 01:30:00-07:00,1.600330e+09,3.100000,0.780459,0.0
3,2020-09-17 09:00:00,2020-09-17 02:00:00-07:00,1.600332e+09,2.766667,0.742278,0.0
4,2020-09-17 09:30:00,2020-09-17 02:30:00-07:00,1.600334e+09,2.966667,0.764326,0.0
...,...,...,...,...,...,...
1421,2020-10-16 22:00:00,2020-10-16 15:00:00-07:00,1.602885e+09,4.200000,0.913958,0.0
1422,2020-10-16 22:30:00,2020-10-16 15:30:00-07:00,1.602886e+09,4.600000,0.950722,0.0
1423,2020-10-16 23:00:00,2020-10-16 16:00:00-07:00,1.602888e+09,4.866667,0.978367,0.0
1424,2020-10-16 23:30:00,2020-10-16 16:30:00-07:00,1.602890e+09,6.433333,1.132818,0.0


In [ ]:
plt.scatter(data1_binned['cpm'],data2_binned['temperature'])
plt.xlabel("CPM - Pocket")
plt.ylabel("Temperature")
plt.show()

In [ ]:
r_counts, p_counts = scipy.stats.pearsonr(data1_binned['cpm'].values, data2_binned['temperature'].values)

r_sqr = r_counts**2
print("The proportion of the variance in the Pocket Geiger described by the D3S is:",r_sqr)
print("corr. coeff. = {}, p-value = {}".format(r_counts, p_counts))

This correlation is about as weak as it can be. But the p-value suggests that it is still a significant correlation. We would expect these to be correlated since they are supposed to be measuring the same thing - radiation in the local environment - and they are in the same location.

It is likely that the very week correlation is related to the large relative uncertainties in the counting statistics for the small counting detector (the Pocket Geiger).

If you go back and rebin to combine statistics - say over an hour or two - you can see this directly. The correlation will get stronger as we reduce the intrinsic relative statistical uncertainties for each set of counts.

In [52]:
test = "111110"

In [53]:
test[0]

'1'

In [54]:
len(test)

6